In [1]:
#1) Upload Dataset
from google.colab import files
uploaded = files.upload()

Saving causal_inference_dataset.csv to causal_inference_dataset.csv


In [2]:
import io
import pandas as pd

fname = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[fname]))
print("File Loaded:", fname)
df.head()

File Loaded: causal_inference_dataset.csv


,treatment,age,income,gender,prior_purchases,outcome
0,0,43,58758,Female,1,4.532956
1,1,58,44610,Male,2,4.772700
2,1,49,58859,Male,2,6.172370
3,1,34,66630,Male,2,4.214793
4,0,31,62307,Female,2,3.699535


In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from joblib import dump

# Feature columns
numeric_features = ['age', 'income', 'prior_purchases']
categorical_features = ['gender']

# Transformers
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Fit the preprocessor on your data
preprocessor.fit(df[numeric_features + categorical_features])

# Save it
dump(preprocessor, "preprocessor.joblib")

print("preprocessor.joblib saved!")

preprocessor.joblib saved!


In [17]:
#2) Basic Info & EDA
df.info()
df.describe(include='all')
print(df['treatment'].value_counts())
print(df['outcome'].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   treatment        500 non-null    int64  
 1   age              500 non-null    int64  
 2   income           500 non-null    int64  
 3   gender           500 non-null    object 
 4   prior_purchases  500 non-null    int64  
 5   outcome          500 non-null    float64
dtypes: float64(1), int64(4), object(1)
memory usage: 23.6+ KB
treatment
1    259
0    241
Name: count, dtype: int64
count    500.000000
mean       4.754900
std        1.753451
min       -0.066945
25%        3.490682
50%        4.783642
75%        5.997776
max        9.813360
Name: outcome, dtype: float64


In [18]:
#preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

treatment_col = 'treatment'
outcome_col = 'outcome'
covariate_cols = [c for c in df.columns if c not in (treatment_col, outcome_col)]

num_cols = df[covariate_cols].select_dtypes(include=['number']).columns.tolist()
cat_cols = [c for c in covariate_cols if c not in num_cols]

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

X = preprocessor.fit_transform(df[covariate_cols])
T = df[treatment_col].values
Y = df[outcome_col].values

print("X Shape:", X.shape)


X Shape: (500, 5)


In [19]:
#Propensity Score Models (Logistic + XGBoost)
from sklearn.linear_model import LogisticRegression
import numpy as np

# Logistic Regression PS Model
logit = LogisticRegression(max_iter=1000)
logit.fit(X, T)
ps_logit = logit.predict_proba(X)[:,1]

# XGBoost PS Model
!pip install xgboost -q
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(eval_metric='logloss')
xgb_clf.fit(X, T)
ps_xgb = xgb_clf.predict_proba(X)[:,1]

print("PS Logit Mean:", ps_logit.mean())
print("PS XGB Mean:", ps_xgb.mean())



PS Logit Mean: 0.5179987856153325
PS XGB Mean: 0.51800764


In [20]:
#Outcome Models (Linear + XGBoost)
from sklearn.linear_model import LinearRegression

# Linear models trained separately
lin0 = LinearRegression()
lin1 = LinearRegression()

lin0.fit(X[T==0], Y[T==0])
lin1.fit(X[T==1], Y[T==1])

mu0_lin = lin0.predict(X)
mu1_lin = lin1.predict(X)

# XGBoost outcome models
xgb_reg0 = xgb.XGBRegressor()
xgb_reg1 = xgb.XGBRegressor()

xgb_reg0.fit(X[T==0], Y[T==0])
xgb_reg1.fit(X[T==1], Y[T==1])

mu0_xgb = xgb_reg0.predict(X)
mu1_xgb = xgb_reg1.predict(X)


In [21]:
#Doubly Robust (AIPW) ATE Function
def aipw_ate(Y, T, e, mu0, mu1):
    eps = 1e-6
    e = np.clip(e, eps, 1-eps)

    # Influence function
    term = ((T - e) / (e*(1-e))) * (Y - (T*mu1 + (1-T)*mu0))
    phi = term + (mu1 - mu0)

    ate = np.mean(phi)
    se = np.std(phi, ddof=1) / np.sqrt(len(phi))
    return ate, se


In [22]:
#ATE Calculation for All Model Combos
results = []

def add_result(ps_name, mu_name, ps, mu0, mu1):
    ate, se = aipw_ate(Y, T, ps, mu0, mu1)
    results.append([ps_name, mu_name, ate, se])

# logit + lin
add_result("logit", "lin", ps_logit, mu0_lin, mu1_lin)

# logit + xgb
add_result("logit", "xgb", ps_logit, mu0_xgb, mu1_xgb)

# xgb + lin
add_result("xgb", "lin", ps_xgb, mu0_lin, mu1_lin)

# xgb + xgb
add_result("xgb", "xgb", ps_xgb, mu0_xgb, mu1_xgb)

df_results = pd.DataFrame(results, columns=["PropensityModel", "OutcomeModel", "ATE", "SE"])
df_results


,PropensityModel,OutcomeModel,ATE,SE
0,logit,lin,1.876395,0.091289
1,logit,xgb,1.871950,0.060545
2,xgb,lin,1.872045,0.058819
3,xgb,xgb,1.871750,0.060256


In [23]:
#Add Confidence Intervals
df_results["CI Lower"] = df_results["ATE"] - 1.96 * df_results["SE"]
df_results["CI Upper"] = df_results["ATE"] + 1.96 * df_results["SE"]
df_results


,PropensityModel,OutcomeModel,ATE,SE,CI Lower,CI Upper
0,logit,lin,1.876395,0.091289,1.697468,2.055321
1,logit,xgb,1.871950,0.060545,1.753282,1.990619
2,xgb,lin,1.872045,0.058819,1.756760,1.987330
3,xgb,xgb,1.871750,0.060256,1.753650,1.989851


In [24]:
#Save Outputs
df_results.to_csv("ate_results.csv", index=False)
print("Saved ate_results.csv")


Saved ate_results.csv


In [25]:
#Final — Show Summary
print("----- FINAL RESULTS -----")
print(df_results)


----- FINAL RESULTS -----
  PropensityModel OutcomeModel       ATE        SE  CI Lower  CI Upper
0           logit          lin  1.876395  0.091289  1.697468  2.055321
1           logit          xgb  1.871950  0.060545  1.753282  1.990619
2             xgb          lin  1.872045  0.058819  1.756760  1.987330
3             xgb          xgb  1.871750  0.060256  1.753650  1.989851
